In [1]:
import numpy as np
from scipy import special

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
import numpy as np
from scipy import special

class Module(object):
    """
    Абстрактный класс модуля нейронной сети.
    Определяет основные методы для прямого и обратного прохода.
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """ Прямой проход """
        return self.updateOutput(input)

    def backward(self, input, gradOutput):
        """ Обратный проход """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput

    def updateOutput(self, input):
        """ Вычисление выхода модуля """
        pass

    def updateGradInput(self, input, gradOutput):
        """ Вычисление градиента по входу """
        pass

    def accGradParameters(self, input, gradOutput):
        """ Вычисление градиента по параметрам модуля (если они есть) """
        pass

    def zeroGradParameters(self):
        """ Обнуление градиентов по параметрам """
        pass

    def getParameters(self):
        """ Получение списка параметров модуля """
        return []

    def getGradParameters(self):
        """ Получение списка градиентов по параметрам """
        return []

    def train(self):
        """ Установка режима обучения """
        self.training = True

    def evaluate(self):
        """ Установка режима оценки """
        self.training = False

    def __repr__(self):
        """ Строковое представление модуля """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [3]:
class Sequential(Module):
    """
    Контейнер для последовательного выполнения модулей.
    """
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
        # Сохраняем входы для каждого слоя для использования в backward pass
        self.inputs = []

    def add(self, module):
        """ Добавление модуля в контейнер """
        self.modules.append(module)

    def updateOutput(self, input):
        """ Последовательный прямой проход по всем модулям """
        self.inputs = [input]
        current_output = input
        for module in self.modules:
            current_output = module.forward(current_output)
            # Сохраняем вход следующего слоя (он же выход текущего)
            self.inputs.append(current_output)
        # Последний сохраненный элемент inputs - это итоговый выход сети
        self.output = self.inputs.pop()
        return self.output

    def updateGradInput(self, input, gradOutput):
        """ Последовательный обратный проход по всем модулям """
        current_grad = gradOutput
        # Идем по слоям с конца, используя сохраненные входы
        for i in range(len(self.modules) - 1, -1, -1):
            # Вход для i-го слоя - это self.inputs[i]
            current_grad = self.modules[i].backward(self.inputs[i], current_grad)
        self.gradInput = current_grad
        return self.gradInput

    def zeroGradParameters(self):
        """ Обнуление градиентов во всех вложенных модулях """
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """ Сбор параметров из всех вложенных модулей """
        params = []
        for module in self.modules:
            module_params = module.getParameters()
            # Проверяем, что возвращается список (даже если там один параметр)
            if isinstance(module_params, list):
                params.extend(module_params)
            # Обработка случая, когда модуль может вернуть None или не список
            elif module_params is not None:
                 # Если не список, но не None, оборачиваем в список
                 # (хотя по контракту Module должен возвращать список)
                 params.append(module_params)

        return params

    def getGradParameters(self):
        """ Сбор градиентов по параметрам из всех вложенных модулей """
        grad_params = []
        for module in self.modules:
            module_grad_params = module.getGradParameters()
             # Проверяем, что возвращается список (даже если там один градиент)
            if isinstance(module_grad_params, list):
                grad_params.extend(module_grad_params)
            # Обработка случая, когда модуль может вернуть None или не список
            elif module_grad_params is not None:
                 grad_params.append(module_grad_params)
        return grad_params

    def __repr__(self):
        """ Строковое представление последовательности модулей """
        string = "".join([repr(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self, x):
        """ Доступ к модулям по индексу """
        return self.modules.__getitem__(x)

    def train(self):
        """ Установка режима обучения для всех вложенных модулей """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """ Установка режима оценки для всех вложенных модулей """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [4]:
class Linear(Module):
    """
    Полносвязный слой (линейное преобразование).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

        # Сохраняем вход для расчета градиентов по параметрам
        self.input_cache = None

    def updateOutput(self, input):
        self.input_cache = input # Сохраняем вход
        self.output = np.dot(input, self.W.T) + self.b
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.dot(gradOutput, self.W)
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # Используем сохраненный input_cache или переданный input
        # (по контракту Module, input должен быть тем же, что и при forward)
        current_input = self.input_cache if self.input_cache is not None else input
        self.gradW += np.dot(gradOutput.T, current_input)
        self.gradb += np.sum(gradOutput, axis=0)

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' % (s[1], s[0])
        return q

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [5]:
class SoftMax(Module):
    """
    Слой SoftMax.
    """
    def __init__(self):
         super(SoftMax, self).__init__()
         # Сохраняем выход для расчета градиента по входу
         self.output_cache = None

    def updateOutput(self, input):
        # Вычитаем максимум для численной стабильности
        input_stable = input - np.max(input, axis=1, keepdims=True)
        exp_input = np.exp(input_stable)
        self.output = exp_input / np.sum(exp_input, axis=1, keepdims=True)
        self.output_cache = self.output # Сохраняем выход
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Используем сохраненный выход
        local_output = self.output_cache if self.output_cache is not None else self.forward(input)

        # gradInput = output * (gradOutput - sum(gradOutput * output))
        sum_grad_output = np.sum(gradOutput * local_output, axis=1, keepdims=True)
        self.gradInput = local_output * (gradOutput - sum_grad_output)
        return self.gradInput

    def __repr__(self):
        return "SoftMax"

## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [6]:
class LogSoftMax(Module):
    """
    Слой LogSoftMax.
    """
    def __init__(self):
         super(LogSoftMax, self).__init__()
         # Сохраняем softmax(input) для расчета градиента
         self.softmax_cache = None

    def updateOutput(self, input):
        # Вычитаем максимум для численной стабильности
        input_stable = input - np.max(input, axis=1, keepdims=True)
        exp_input = np.exp(input_stable)
        sum_exp = np.sum(exp_input, axis=1, keepdims=True)
        log_sum_exp = np.log(sum_exp)

        self.output = input_stable - log_sum_exp
        # Сохраняем softmax для backward pass
        self.softmax_cache = exp_input / sum_exp
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Используем сохраненный softmax
        softmax_output = self.softmax_cache
        if softmax_output is None: # Пересчитываем, если не сохранен
             input_stable = input - np.max(input, axis=1, keepdims=True)
             exp_input = np.exp(input_stable)
             softmax_output = exp_input / np.sum(exp_input, axis=1, keepdims=True)

        # gradInput = gradOutput - softmax(x) * sum(gradOutput)
        sum_grad_output = np.sum(gradOutput, axis=1, keepdims=True)
        self.gradInput = gradOutput - softmax_output * sum_grad_output
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [7]:
class BatchNormalization(Module):
    """
    Слой Batch Normalization.
    """
    EPS = 1e-5 # Используем стандартный epsilon как в PyTorch
    # Переименовали alpha в momentum и установили стандартное значение PyTorch
    def __init__(self, num_features, momentum = 0.1):
        super(BatchNormalization, self).__init__()
        self.momentum = momentum # Это эквивалент momentum в PyTorch
        # Параметры для статистики по батчу
        self.moving_mean = np.zeros(num_features)
        self.moving_variance = np.ones(num_features) # Инициализируем единицами

        # Кэш для backward pass
        self.input_reshaped = None
        self.x_centered = None
        self.std_inv = None
        self.normalized = None
        self.input_shape = None
        self.num_features = num_features

    def _reshape_input(self, input):
         """ Вспомогательная функция для изменения формы входа """
         self.input_shape = input.shape
         if len(self.input_shape) == 2: # (N, C)
             return input, self.input_shape[1]
         elif len(self.input_shape) == 4: # (N, C, H, W)
             N, C, H, W = self.input_shape
             # Преобразуем к (N*H*W, C) для вычисления статистики по каналам
             return input.transpose(0, 2, 3, 1).reshape(-1, C), C
         else:
             raise ValueError("Input shape not supported for BatchNormalization")

    def _reshape_output(self, output_reshaped):
         """ Вспомогательная функция для возвращения формы выхода """
         if len(self.input_shape) == 4:
             N, C, H, W = self.input_shape
             # Преобразуем обратно к (N, C, H, W)
             return output_reshaped.reshape(N, H, W, C).transpose(0, 3, 1, 2)
         return output_reshaped # Для 2D входа форма не меняется

    def updateOutput(self, input):
        input_reshaped, num_features = self._reshape_input(input)
        if num_features != self.num_features:
             raise ValueError(f"Expected {self.num_features} features, but got {num_features}")


        if self.training:
            batch_mean = np.mean(input_reshaped, axis=0)
            # Используем ddof=0 для смещенной оценки дисперсии, как в PyTorch при обучении
            batch_variance = np.var(input_reshaped, axis=0, ddof=0)
            self.std_inv = 1. / np.sqrt(batch_variance + self.EPS)
            self.x_centered = input_reshaped - batch_mean
            self.normalized = self.x_centered * self.std_inv

            # Обновляем скользящие средние
            # Формула PyTorch: running_mean = (1 - momentum) * running_mean + momentum * batch_mean
            self.moving_mean = (1.0 - self.momentum) * self.moving_mean + self.momentum * batch_mean

            # PyTorch обновляет variance с использованием несмещенной оценки
            n = input_reshaped.shape[0]
            # Смещенная дисперсия батча используется для нормализации выхода,
            # но несмещенная (если n > 1) используется для обновления moving_variance
            unbiased_batch_variance = batch_variance * n / (n - 1) if n > 1 else batch_variance
            self.moving_variance = (1.0 - self.momentum) * self.moving_variance + self.momentum * unbiased_batch_variance

            output_reshaped = self.normalized
        else:
            # Используем скользящие средние и дисперсию
            output_reshaped = (input_reshaped - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)

        self.output = self._reshape_output(output_reshaped)
        return self.output

    def updateGradInput(self, input, gradOutput):
        gradOutput_reshaped, _ = self._reshape_input(gradOutput) # Приводим gradOutput к форме (N*H*W, C) или (N, C)
        input_reshaped, _ = self._reshape_input(input) # Нужен решейпнутый инпут для градиентов

        if self.training:
            N = gradOutput_reshaped.shape[0]
            if N == 1: # Градиенты для batch_size=1 немного другие (var=0)
                # В этом случае dL/dvar и часть dL/dmu обнуляются
                dvar = np.zeros_like(np.mean(input_reshaped, axis=0))
                dmu = np.sum(gradOutput_reshaped * -self.std_inv, axis=0)
                gradInput_reshaped = gradOutput_reshaped * self.std_inv + dmu / N
            else:
                # Шаги обратного прохода для BN (стандартные формулы)
                # dL/dx_norm = gradOutput_reshaped
                # dL/dvar = sum(dL/dx_norm * x_centered * (-0.5) * std_inv^3)
                dvar = np.sum(gradOutput_reshaped * self.x_centered * -0.5 * self.std_inv**3, axis=0)

                # dL/dmean = sum(dL/dx_norm * (-std_inv)) + dL/dvar * sum(-2 * x_centered) / N
                # Используем sum вместо mean для части с dvar, т.к. N уже учтено ниже
                dmu = np.sum(gradOutput_reshaped * -self.std_inv, axis=0) + \
                      dvar * np.sum(-2. * self.x_centered, axis=0) / N

                # dL/dx = dL/dx_norm * std_inv + dL/dvar * (2 * x_centered / N) + dL/dmean / N
                gradInput_reshaped = gradOutput_reshaped * self.std_inv + \
                                    dvar * 2. * self.x_centered / N + \
                                    dmu / N
        else:
            # При оценке градиент просто масштабируется
            std_eval_inv = 1. / np.sqrt(self.moving_variance + self.EPS)
            gradInput_reshaped = gradOutput_reshaped * std_eval_inv

        self.gradInput = self._reshape_output(gradInput_reshaped)
        return self.gradInput

    def __repr__(self):
        # Отображаем momentum вместо alpha
        return f"BatchNormalization(num_features={self.num_features}, momentum={self.momentum})"

In [8]:
class ChannelwiseScaling(Module):
    """
    Масштабирование и сдвиг по каналам (аффинное преобразование в BatchNorm).
    """
    def __init__(self, num_features):
        super(ChannelwiseScaling, self).__init__()

        # Инициализация как в PyTorch BatchNorm (gamma=1, beta=0)
        self.gamma = np.ones(num_features)
        self.beta = np.zeros(num_features)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

        # Кэш
        self.input_cache = None
        self.input_shape = None
        self.num_features = num_features

    def _reshape_params_grads(self, tensor):
        """ Придает нужную форму параметрам/градиентам для broadcasting """
        if len(self.input_shape) == 4: # (N, C, H, W)
             # Форма (1, C, 1, 1) для broadcasting по N, H, W
             return tensor.reshape(1, -1, 1, 1)
        elif len(self.input_shape) == 2: # (N, C)
             # Форма (1, C) для broadcasting по N
             return tensor.reshape(1, -1)
        return tensor # Не должно случиться

    def _sum_grads(self, grad):
         """ Суммирует градиенты по всем осям, кроме канальной """
         if len(self.input_shape) == 4: # (N, C, H, W) -> sum over N, H, W
             return np.sum(grad, axis=(0, 2, 3))
         elif len(self.input_shape) == 2: # (N, C) -> sum over N
             return np.sum(grad, axis=0)
         return grad

    def updateOutput(self, input):
        self.input_cache = input
        self.input_shape = input.shape
        assert input.shape[1] == self.num_features # Проверка каналов

        gamma_reshaped = self._reshape_params_grads(self.gamma)
        beta_reshaped = self._reshape_params_grads(self.beta)

        self.output = input * gamma_reshaped + beta_reshaped
        return self.output

    def updateGradInput(self, input, gradOutput):
        gamma_reshaped = self._reshape_params_grads(self.gamma)
        self.gradInput = gradOutput * gamma_reshaped
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # Используем input_cache или переданный input
        current_input = self.input_cache if self.input_cache is not None else input
        self.gradBeta += self._sum_grads(gradOutput)
        self.gradGamma += self._sum_grads(gradOutput * current_input)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return f"ChannelwiseScaling(num_features={self.num_features})"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [9]:
class Dropout(Module):
    """
    Слой Dropout.
    """
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        if not (0.0 <= p < 1.0):
             raise ValueError("dropout probability has to be between 0 and 1, but got {}".format(p))
        self.p = p
        self.mask = None
        # Масштабирующий множитель применяется при обучении
        self.scale = 1.0 / (1.0 - self.p) if self.p < 1.0 else 0.0

    def updateOutput(self, input):
        if self.training:
            # Генерируем маску: 1 с вероятностью (1-p), 0 с вероятностью p
            self.mask = (np.random.rand(*input.shape) > self.p).astype(input.dtype)
            # Применяем маску и масштабируем (inverted dropout)
            self.output = input * self.mask * self.scale
        else:
            # В режиме оценки ничего не делаем
            self.output = input
            self.mask = None # Маска не нужна при оценке
        return self.output

    def updateGradInput(self, input, gradOutput):
        if self.training:
            # Градиент проходит только там, где mask=1, и масштабируется
            if self.mask is None: # Если backward вызван без forward в режиме train
                 raise RuntimeError("Cannot call backward in training mode without calling forward first")
            self.gradInput = gradOutput * self.mask * self.scale
        else:
            # В режиме оценки градиент проходит без изменений
            self.gradInput = gradOutput
        return self.gradInput

    def __repr__(self):
        return f"Dropout(p={self.p})"

# 6. (2.0) Conv2d
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

In [10]:
def im2col_indices(x, kernel_h, kernel_w, stride_h=1, stride_w=1, pad_h=0, pad_w=0):
    """ Преобразует изображение в столбцы (матрицу патчей) """
    N, C, H, W = x.shape
    out_h = (H + 2 * pad_h - kernel_h) // stride_h + 1
    out_w = (W + 2 * pad_w - kernel_w) // stride_w + 1

    # Добавляем padding
    img = np.pad(x, [(0,0), (0,0), (pad_h, pad_h), (pad_w, pad_w)], mode='constant')

    # Вычисляем индексы для каждого патча
    i0 = np.repeat(np.arange(kernel_h), kernel_w)
    i0 = np.tile(i0, C)
    i1 = stride_h * np.repeat(np.arange(out_h), out_w)
    j0 = np.tile(np.arange(kernel_w), kernel_h * C)
    j1 = stride_w * np.tile(np.arange(out_w), out_h)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)

    k = np.repeat(np.arange(C), kernel_h * kernel_w).reshape(-1, 1)

    # Извлекаем патчи с помощью индексов
    cols = img[:, k, i, j] # Shape (N, C*kH*kW, outH*outW)
    cols = cols.transpose(1, 2, 0).reshape(C * kernel_h * kernel_w, -1) # Shape (C*kH*kW, N*outH*outW)
    return cols


def col2im_indices(cols, x_shape, kernel_h, kernel_w, stride_h=1, stride_w=1, pad_h=0, pad_w=0):
    """ Преобразует столбцы обратно в изображение """
    N, C, H, W = x_shape
    H_padded, W_padded = H + 2 * pad_h, W + 2 * pad_w
    out_h = (H + 2 * pad_h - kernel_h) // stride_h + 1
    out_w = (W + 2 * pad_w - kernel_w) // stride_w + 1

    # Создаем пустой тензор для изображения с паддингом
    img = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)

    # Вычисляем индексы так же, как в im2col
    i0 = np.repeat(np.arange(kernel_h), kernel_w)
    i0 = np.tile(i0, C)
    i1 = stride_h * np.repeat(np.arange(out_h), out_w)
    j0 = np.tile(np.arange(kernel_w), kernel_h * C)
    j1 = stride_w * np.tile(np.arange(out_w), out_h)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    k = np.repeat(np.arange(C), kernel_h * kernel_w).reshape(-1, 1)

    # Преобразуем cols обратно к (C*kH*kW, N*outH*outW) -> (N, C*kH*kW, outH*outW)
    cols_reshaped = cols.reshape(C * kernel_h * kernel_w, -1, N)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)

    # Используем np.add.at для аккуратного сложения градиентов в нужные ячейки
    # Это важно, если патчи перекрываются (stride < kernel_size)
    np.add.at(img, (slice(None), k, i, j), cols_reshaped)

    # Убираем padding
    if pad_h > 0 or pad_w > 0:
        return img[:, :, pad_h:H_padded-pad_h, pad_w:W_padded-pad_w]
    return img

# --- Конец утилит для Conv2d ---

class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True, padding_mode='zeros'):
        super(Conv2d, self).__init__()
        # Проверка padding_mode пока убрана, т.к. im2col/col2im поддерживают только 'constant' (нули)
        if padding_mode != 'zeros':
             print(f"Warning: Conv2d padding_mode='{padding_mode}' is not fully supported by current im2col/col2im; using zero padding.")
             self.padding_mode = 'zeros'
        else:
            self.padding_mode = padding_mode

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        self.padding = (padding, padding) if isinstance(padding, int) else padding

        # Инициализация весов (He initialization)
        stdv = np.sqrt(2.0 / (self.in_channels * self.kernel_size[0] * self.kernel_size[1]))
        self.W = np.random.normal(0, stdv, size=(self.out_channels, self.in_channels, self.kernel_size[0], self.kernel_size[1]))
        self.gradW = np.zeros_like(self.W)

        self.bias = bias
        if bias:
            # Инициализация смещения нулями
            self.b = np.zeros(self.out_channels)
            self.gradb = np.zeros_like(self.b)
        else:
            self.b = None
            self.gradb = None

        # Кэш
        self.input_shape = None
        self.input_col = None # Сохраняем вход в виде столбцов

    def updateOutput(self, input):
        self.input_shape = input.shape
        N, C, H, W = self.input_shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding

        # Вычисляем размеры выхода
        out_H = (H + 2 * pH - kH) // sH + 1
        out_W = (W + 2 * pW - kW) // sW + 1

        # Преобразуем вход в столбцы
        self.input_col = im2col_indices(input, kH, kW, sH, sW, pH, pW)
        # Преобразуем веса в строки
        W_rows = self.W.reshape(self.out_channels, -1)

        # Выполняем свертку как матричное умножение
        output_col = np.dot(W_rows, self.input_col) # (out_C, N*out_H*out_W)

        # Добавляем смещение, если есть
        if self.bias:
            output_col += self.b.reshape(-1, 1) # (out_C, 1) broadcasts

        # Преобразуем результат обратно в 4D тензор (out_C, N, out_H, out_W) -> (N, out_C, out_H, out_W)
        self.output = output_col.reshape(self.out_channels, out_H, out_W, N).transpose(3, 0, 1, 2)

        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = self.input_shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding
        _, out_C, out_H, out_W = gradOutput.shape

        # Преобразуем gradOutput к формату столбцов (out_C, N*out_H*out_W)
        gradOutput_col = gradOutput.transpose(1, 2, 3, 0).reshape(self.out_channels, -1)

        # Преобразуем веса (out_C, in_C*kH*kW)
        W_rows = self.W.reshape(self.out_channels, -1)

        # Вычисляем градиент по входу (в формате столбцов)
        # gradInput_col = W^T @ gradOutput_col
        gradInput_col = np.dot(W_rows.T, gradOutput_col) # (in_C*kH*kW, N*out_H*out_W)

        # Преобразуем градиент из столбцов обратно в изображение
        self.gradInput = col2im_indices(gradInput_col, self.input_shape, kH, kW, sH, sW, pH, pW)

        return self.gradInput

    def accGradParameters(self, input, gradOutput):
         N, C, H, W = self.input_shape
         kH, kW = self.kernel_size
         sH, sW = self.stride
         pH, pW = self.padding
         _, out_C, out_H, out_W = gradOutput.shape

         # Преобразуем gradOutput к формату столбцов (out_C, N*out_H*out_W)
         gradOutput_col = gradOutput.transpose(1, 2, 3, 0).reshape(self.out_channels, -1)

         # Градиент по весам: gradW = gradOutput_col @ input_col^T
         # input_col имеет форму (in_C*kH*kW, N*out_H*out_W)
         # Нам нужен input_col из forward pass
         if self.input_col is None:
              # Если backward вызван без forward, пересчитываем input_col
              # (это не должно происходить при стандартном использовании)
              current_input_col = im2col_indices(input, kH, kW, sH, sW, pH, pW)
         else:
             current_input_col = self.input_col

         gradW_flat = np.dot(gradOutput_col, current_input_col.T) # (out_C, in_C*kH*kW)
         self.gradW += gradW_flat.reshape(self.W.shape) # Приводим к исходной форме и накапливаем

         # Градиент по смещению: gradb = sum(gradOutput_col) по оси батча (N*out_H*out_W)
         if self.bias:
             self.gradb += np.sum(gradOutput_col, axis=1) # Суммируем по всем N*out_H*out_W элементам для каждого out_C

    def zeroGradParameters(self):
        self.gradW.fill(0)
        if self.bias:
            self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b] if self.bias else [self.W]

    def getGradParameters(self):
        return [self.gradW, self.gradb] if self.bias else [self.gradW]

    def __repr__(self):
        return (f"Conv2d(in_channels={self.in_channels}, out_channels={self.out_channels}, "
                f"kernel_size={self.kernel_size}, stride={self.stride}, padding={self.padding}, "
                f"bias={self.bias})") # Убрал padding_mode, так как пока только 'zeros'

# 7. (0.5) Implement [**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

In [11]:
class MaxPool2d(Module):
    def __init__(self, kernel_size, stride=None, padding=0):
        super(MaxPool2d, self).__init__()
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        # Если stride не задан, он равен kernel_size (стандартное поведение)
        self.stride = (stride, stride) if isinstance(stride, int) else \
                      stride if stride is not None else self.kernel_size
        self.padding = (padding, padding) if isinstance(padding, int) else padding

        # Кэш для backward pass
        self.input_shape = None
        self.indices = None # Сохраняем индексы максимальных элементов

    def updateOutput(self, input):
        self.input_shape = input.shape
        N, C, H, W = self.input_shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding

        # Добавляем padding (отрицательная бесконечность, чтобы не влиять на max)
        if pH > 0 or pW > 0:
            input_padded = np.pad(input, ((0, 0), (0, 0), (pH, pH), (pW, pW)),
                                  mode='constant', constant_values=-np.inf)
            H_pad, W_pad = H + 2*pH, W + 2*pW
        else:
            input_padded = input
            H_pad, W_pad = H, W

        # Вычисляем размеры выхода
        out_H = (H_pad - kH) // sH + 1
        out_W = (W_pad - kW) // sW + 1

        # Создаем тензоры для выхода и индексов
        self.output = np.zeros((N, C, out_H, out_W), dtype=input.dtype)
        self.indices = np.zeros((N, C, out_H, out_W), dtype=int) # Храним плоские индексы внутри окна

        # Итеративный подход (медленный, но понятный)
        for n in range(N):
            for c in range(C):
                for h in range(out_H):
                    for w in range(out_W):
                        # Определяем границы текущего окна в input_padded
                        h_start, w_start = h * sH, w * sW
                        h_end, w_end = h_start + kH, w_start + kW
                        window = input_padded[n, c, h_start:h_end, w_start:w_end]

                        # Находим максимум и его плоский индекс в окне
                        self.output[n, c, h, w] = np.max(window)
                        self.indices[n, c, h, w] = np.argmax(window)

        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = self.input_shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding
        _, _, out_H, out_W = gradOutput.shape

        # Создаем тензор для градиента по входу с паддингом
        H_pad, W_pad = H + 2*pH, W + 2*pW
        gradInput_padded = np.zeros((N, C, H_pad, W_pad), dtype=gradOutput.dtype)

        # Распространяем градиенты по индексам
        for n in range(N):
            for c in range(C):
                for h in range(out_H):
                    for w in range(out_W):
                        # Получаем плоский индекс максимума в окне
                        flat_idx = self.indices[n, c, h, w]
                        # Преобразуем плоский индекс в 2D координаты (h_idx, w_idx) внутри окна
                        h_idx = flat_idx // kW
                        w_idx = flat_idx % kW

                        # Определяем абсолютные координаты в gradInput_padded
                        h_start, w_start = h * sH, w * sW
                        abs_h_idx = h_start + h_idx
                        abs_w_idx = w_start + w_idx

                        # Добавляем градиент (+= т.к. окна могут перекрываться)
                        gradInput_padded[n, c, abs_h_idx, abs_w_idx] += gradOutput[n, c, h, w]

        # Убираем padding
        if pH > 0 or pW > 0:
            self.gradInput = gradInput_padded[:, :, pH:-pH if pH > 0 else None, pW:-pW if pW > 0 else None]
        else:
            self.gradInput = gradInput_padded

        return self.gradInput

    def __repr__(self):
        return f"MaxPool2d(kernel_size={self.kernel_size}, stride={self.stride}, padding={self.padding})"

class AvgPool2d(Module):
    def __init__(self, kernel_size, stride=None, padding=0):
        super(AvgPool2d, self).__init__()
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else \
                      stride if stride is not None else self.kernel_size
        self.padding = (padding, padding) if isinstance(padding, int) else padding

        # Кэш
        self.input_shape = None
        self.window_size = self.kernel_size[0] * self.kernel_size[1]

    def updateOutput(self, input):
        self.input_shape = input.shape
        N, C, H, W = self.input_shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding

        # Добавляем padding (нули)
        if pH > 0 or pW > 0:
            input_padded = np.pad(input, ((0, 0), (0, 0), (pH, pH), (pW, pW)), mode='constant')
            H_pad, W_pad = H + 2*pH, W + 2*pW
        else:
            input_padded = input
            H_pad, W_pad = H, W

        # Вычисляем размеры выхода
        out_H = (H_pad - kH) // sH + 1
        out_W = (W_pad - kW) // sW + 1

        self.output = np.zeros((N, C, out_H, out_W), dtype=input.dtype)

        # Итеративный подход
        for n in range(N):
            for c in range(C):
                for h in range(out_H):
                    for w in range(out_W):
                        h_start, w_start = h * sH, w * sW
                        h_end, w_end = h_start + kH, w_start + kW
                        window = input_padded[n, c, h_start:h_end, w_start:w_end]
                        self.output[n, c, h, w] = np.mean(window)

        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = self.input_shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding
        _, _, out_H, out_W = gradOutput.shape

        # Создаем тензор для градиента по входу с паддингом
        H_pad, W_pad = H + 2*pH, W + 2*pW
        gradInput_padded = np.zeros((N, C, H_pad, W_pad), dtype=gradOutput.dtype)

        # Масштабирующий коэффициент для градиента
        scale = 1.0 / self.window_size

        # Распределяем градиенты равномерно по окну
        for n in range(N):
            for c in range(C):
                for h in range(out_H):
                    for w in range(out_W):
                        h_start, w_start = h * sH, w * sW
                        h_end, w_end = h_start + kH, w_start + kW
                        # Добавляем масштабированный градиент ко всем элементам окна
                        gradInput_padded[n, c, h_start:h_end, w_start:w_end] += \
                            gradOutput[n, c, h, w] * scale

        # Убираем padding
        if pH > 0 or pW > 0:
             self.gradInput = gradInput_padded[:, :, pH:-pH if pH > 0 else None, pW:-pW if pW > 0 else None]
        else:
             self.gradInput = gradInput_padded

        return self.gradInput

    def __repr__(self):
        return f"AvgPool2d(kernel_size={self.kernel_size}, stride={self.stride}, padding={self.padding})"

# 8. (0.3) Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**. They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [12]:
class GlobalMaxPool2d(Module):
    """
    Global Max Pooling по пространственным измерениям (H, W).
    """
    def __init__(self):
        super(GlobalMaxPool2d, self).__init__()
        # Кэш
        self.input_shape = None
        self.max_mask = None # Маска для хранения позиций максимумов

    def updateOutput(self, input):
        self.input_shape = input.shape
        N, C, H, W = self.input_shape

        # Находим максимум по осям H и W, сохраняя размерность C
        self.output = np.max(input, axis=(2, 3), keepdims=True) # Shape (N, C, 1, 1)

        # Создаем маску, где 1 соответствуют максимальным значениям
        # Сравниваем input с output, расширенным до формы input
        self.max_mask = (input == self.output)

        # Убираем лишние оси (1, 1) для соответствия выходу (N, C)
        self.output = self.output.reshape(N, C)
        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C = gradOutput.shape
        H, W = self.input_shape[2:]

        # Расширяем gradOutput до формы (N, C, 1, 1) для broadcasting
        gradOutput_expanded = gradOutput.reshape(N, C, 1, 1)

        # Создаем градиент по входу
        # Градиент равен gradOutput там, где был максимум, и 0 иначе
        # Делим на количество максимумов, если их несколько в одном срезе (редко, но возможно)
        num_maxima = np.sum(self.max_mask, axis=(2, 3), keepdims=True)
        num_maxima[num_maxima == 0] = 1 # Избегаем деления на ноль

        self.gradInput = (self.max_mask * gradOutput_expanded) / num_maxima
        return self.gradInput

    def __repr__(self):
        return "GlobalMaxPool2d"

class GlobalAvgPool2d(Module):
    """
    Global Average Pooling по пространственным измерениям (H, W).
    """
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
        # Кэш
        self.input_shape = None
        self.spatial_size = None # H * W

    def updateOutput(self, input):
        self.input_shape = input.shape
        N, C, H, W = self.input_shape
        self.spatial_size = H * W

        # Вычисляем среднее по осям H и W
        self.output = np.mean(input, axis=(2, 3)) # Shape (N, C)
        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C = gradOutput.shape
        H, W = self.input_shape[2:]

        # Расширяем gradOutput до формы (N, C, 1, 1)
        gradOutput_expanded = gradOutput.reshape(N, C, 1, 1)

        # Распределяем градиент равномерно по пространственным измерениям
        # Каждый элемент входа получает gradOutput / (H * W)
        self.gradInput = np.ones_like(input) * gradOutput_expanded / self.spatial_size
        return self.gradInput

    def __repr__(self):
        return "GlobalAvgPool2d"

# Тестовая функция
def test_global_pooling():
    batch_size = 2
    channels = 3
    height = 4
    width = 5 # Не квадратное для теста
    input_data = np.arange(batch_size * channels * height * width).reshape(batch_size, channels, height, width).astype(np.float32)
    input_data += np.random.randn(batch_size, channels, height, width) * 0.1 # Добавим шум

    print("--- GlobalMaxPool2d Test ---")
    gmax_pool = GlobalMaxPool2d()
    output_max = gmax_pool.forward(input_data)
    print(f"Input shape: {input_data.shape}")
    # Ожидаемый выход: максимум по последним двум осям
    expected_output_max = np.max(input_data, axis=(2, 3))
    print(f"Output shape: {output_max.shape}, Expected shape: {expected_output_max.shape}")
    print(f"Output matches expected: {np.allclose(output_max, expected_output_max)}")
    # Тест градиента
    grad_output_max = np.random.randn(*output_max.shape)
    grad_input_max = gmax_pool.backward(input_data, grad_output_max)
    print(f"Gradient input shape: {grad_input_max.shape}")
    # Проверка: градиент должен быть ненулевым только в позициях максимумов
    non_zero_grads = grad_input_max[gmax_pool.max_mask]
    zero_grads = grad_input_max[~gmax_pool.max_mask]
    print(f"All zero-masked grads are zero: {np.all(zero_grads == 0)}")
    print(f"Number of non-zero grads: {len(non_zero_grads)}")


    print("\n--- GlobalAvgPool2d Test ---")
    gavg_pool = GlobalAvgPool2d()
    output_avg = gavg_pool.forward(input_data)
    print(f"Input shape: {input_data.shape}")
    # Ожидаемый выход: среднее по последним двум осям
    expected_output_avg = np.mean(input_data, axis=(2, 3))
    print(f"Output shape: {output_avg.shape}, Expected shape: {expected_output_avg.shape}")
    print(f"Output matches expected: {np.allclose(output_avg, expected_output_avg)}")
    # Тест градиента
    grad_output_avg = np.random.randn(*output_avg.shape)
    grad_input_avg = gavg_pool.backward(input_data, grad_output_avg)
    print(f"Gradient input shape: {grad_input_avg.shape}")
    # Проверка: градиент должен быть равен gradOutput / (H*W) для всех элементов
    expected_grad_val = grad_output_avg.reshape(batch_size, channels, 1, 1) / (height * width)
    print(f"Gradient input matches expected: {np.allclose(grad_input_avg, expected_grad_val)}")


test_global_pooling()

--- GlobalMaxPool2d Test ---
Input shape: (2, 3, 4, 5)
Output shape: (2, 3), Expected shape: (2, 3)
Output matches expected: True
Gradient input shape: (2, 3, 4, 5)
All zero-masked grads are zero: True
Number of non-zero grads: 6

--- GlobalAvgPool2d Test ---
Input shape: (2, 3, 4, 5)
Output shape: (2, 3), Expected shape: (2, 3)
Output matches expected: True
Gradient input shape: (2, 3, 4, 5)
Gradient input matches expected: True


# 9. (0.2) Implement [**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

In [13]:
class Flatten(Module):
    """
    Слой Flatten, преобразующий многомерный тензор в 2D (батч, признаки).
    Стандартно "сплющивает" все оси, кроме первой (батч).
    """
    def __init__(self):
        super(Flatten, self).__init__()
        # Кэш
        self.input_shape = None

    def updateOutput(self, input):
        self.input_shape = input.shape
        batch_size = self.input_shape[0]
        # Вычисляем размерность признаков как произведение всех остальных размерностей
        num_features = np.prod(self.input_shape[1:])
        # Преобразуем к (batch_size, num_features)
        self.output = input.reshape(batch_size, num_features)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Восстанавливаем исходную форму градиента
        self.gradInput = gradOutput.reshape(self.input_shape)
        return self.gradInput

    def __repr__(self):
        return "Flatten"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [14]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
         # Кэш
         self.input_cache = None

    def updateOutput(self, input):
        self.input_cache = input # Сохраняем вход для backward
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Используем сохраненный input или переданный
        current_input = self.input_cache if self.input_cache is not None else input
        # Градиент = gradOutput там, где input > 0, и 0 иначе
        self.gradInput = np.multiply(gradOutput, current_input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [15]:
class LeakyReLU(Module):
    def __init__(self, slope=0.01): # Стандартный slope в PyTorch 0.01
        super(LeakyReLU, self).__init__()
        self.slope = slope
        # Кэш
        self.input_cache = None

    def updateOutput(self, input):
        self.input_cache = input
        self.output = np.maximum(input, self.slope * input)
        # Или можно использовать np.where:
        # self.output = np.where(input > 0, input, self.slope * input)
        return self.output

    def updateGradInput(self, input, gradOutput):
        current_input = self.input_cache if self.input_cache is not None else input
        # Градиент = 1 где input > 0, slope иначе
        grad_mask = np.where(current_input > 0, 1.0, self.slope)
        self.gradInput = gradOutput * grad_mask
        return self.gradInput

    def __repr__(self):
        return f"LeakyReLU(slope={self.slope})"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [16]:
class ELU(Module):
    def __init__(self, alpha=1.0):
        super(ELU, self).__init__()
        self.alpha = alpha
        # Кэш
        self.input_cache = None
        self.exp_cache = None # Кэшируем exp(x) для отрицательных x

    def updateOutput(self, input):
        self.input_cache = input
        # ELU(x) = x if x > 0 else alpha * (exp(x) - 1)
        positive_mask = input > 0
        negative_input = input[~positive_mask]
        self.exp_cache = np.exp(negative_input) # Кэшируем exp для backward

        self.output = np.empty_like(input)
        self.output[positive_mask] = input[positive_mask]
        self.output[~positive_mask] = self.alpha * (self.exp_cache - 1)
        return self.output

    def updateGradInput(self, input, gradOutput):
        current_input = self.input_cache if self.input_cache is not None else input
        positive_mask = current_input > 0

        # Градиент = 1 если x > 0, alpha * exp(x) иначе
        grad_mask = np.ones_like(current_input)
        # Используем кэшированное значение exp(x)
        if self.exp_cache is not None and np.any(~positive_mask):
             grad_mask[~positive_mask] = self.alpha * self.exp_cache
        elif np.any(~positive_mask): # Если кэш не сработал (backward без forward)
             grad_mask[~positive_mask] = self.alpha * np.exp(current_input[~positive_mask])

        self.gradInput = gradOutput * grad_mask
        return self.gradInput

    def __repr__(self):
        return f"ELU(alpha={self.alpha})"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [17]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()
        # Кэш
        self.input_cache = None

    def updateOutput(self, input):
        self.input_cache = input
        # SoftPlus(x) = log(1 + exp(x))
        # Используем log1p для большей точности при малых exp(x)
        self.output = np.log1p(np.exp(input))
        return self.output

    def updateGradInput(self, input, gradOutput):
        current_input = self.input_cache if self.input_cache is not None else input
        # Градиент = 1 / (1 + exp(-x)) = sigmoid(x)
        sigmoid_input = 1.0 / (1.0 + np.exp(-current_input))
        self.gradInput = gradOutput * sigmoid_input
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

# 13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [18]:
class Gelu(Module):
    """
    Gaussian Error Linear Unit (GELU) активация.
    Использует аппроксимацию через erf.
    """
    def __init__(self):
        super(Gelu, self).__init__()
        # Кэш для backward pass
        self.input_cache = None
        self.cdf_cache = None
        self.pdf_cache = None

    def updateOutput(self, input):
        self.input_cache = input
        # GELU(x) = 0.5 * x * (1 + erf(x / sqrt(2)))
        # erf - error function из scipy.special
        arg = input / np.sqrt(2.0)
        self.cdf_cache = 0.5 * (1.0 + special.erf(arg)) # CDF стандартного нормального распределения
        self.output = 0.5 * input * (1.0 + special.erf(arg))
        # Кэшируем PDF для градиента
        self.pdf_cache = np.exp(-0.5 * input**2) / np.sqrt(2.0 * np.pi)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Используем кэшированные значения
        current_input = self.input_cache
        cdf = self.cdf_cache
        pdf = self.pdf_cache

        # Если кэш пуст (backward без forward)
        if current_input is None or cdf is None or pdf is None:
            current_input = input
            arg = current_input / np.sqrt(2.0)
            cdf = 0.5 * (1.0 + special.erf(arg))
            pdf = np.exp(-0.5 * current_input**2) / np.sqrt(2.0 * np.pi)

        # Градиент GELU: dGELU/dx = CDF(x) + x * PDF(x)
        grad_gelu = cdf + current_input * pdf
        self.gradInput = gradOutput * grad_gelu
        return self.gradInput

    def __repr__(self):
        return "Gelu"

# Criterions

Criterions are used to score the models answers.

In [19]:
class Criterion(object):
    """
    Абстрактный класс для функций потерь.
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """ Прямой проход (вычисление потерь) """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """ Обратный проход (вычисление градиента по входу функции потерь) """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """ Вычисление значения функции потерь """
        pass

    def updateGradInput(self, input, target):
        """ Вычисление градиента функции потерь по ее входу (input) """
        pass

    def __repr__(self):
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [20]:
class MSECriterion(Criterion):
    """
    Mean Squared Error Loss (Среднеквадратичная ошибка).
    """
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        # MSE = sum((input - target)^2) / N
        # Где N - размер батча
        batch_size = input.shape[0]
        self.output = np.sum(np.power(input - target, 2)) / batch_size
        return self.output

    def updateGradInput(self, input, target):
        # gradInput = d(MSE)/d(input) = 2 * (input - target) / N
        batch_size = input.shape[0]
        self.gradInput = 2.0 * (input - target) / batch_size
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [21]:
class ClassNLLCriterionUnstable(Criterion):
    """
    Negative Log Likelihood Loss (нестабильная версия).
    Принимает на вход вероятности (выход SoftMax).
    Требует target в формате one-hot encoding.
    """
    EPS = 1e-15 # Малое число для избежания log(0)
    def __init__(self):
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):
        batch_size = input.shape[0]
        # Обрезаем вероятности, чтобы избежать log(0)
        input_clamp = np.clip(input, self.EPS, 1.0 - self.EPS)

        # NLL = -1/N * sum_i sum_c (target_ic * log(input_ic))
        # Для one-hot target, sum_c оставляет только log(p_k), где k - верный класс
        # Умножение на target выбирает нужные логарифмы
        log_likelihood = np.sum(target * np.log(input_clamp), axis=1)
        self.output = -np.mean(log_likelihood) # Среднее по батчу
        return self.output

    def updateGradInput(self, input, target):
        batch_size = input.shape[0]
        # Обрезаем вероятности так же, как в forward
        input_clamp = np.clip(input, self.EPS, 1.0 - self.EPS)

        # gradInput = d(NLL)/d(input) = -1/N * target / input_clamp
        self.gradInput = - target / (input_clamp * batch_size)
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [22]:
class ClassNLLCriterion(Criterion):
    """
    Negative Log Likelihood Loss (стабильная версия).
    Принимает на вход логарифмы вероятностей (выход LogSoftMax).
    Требует target в формате one-hot encoding.
    """
    def __init__(self):
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
        batch_size = input.shape[0]
        # input - это log(p)
        # NLL = -1/N * sum_i sum_c (target_ic * input_ic)
        # Для one-hot target, sum_c оставляет только log(p_k) для верного класса k
        # Умножение на target выбирает нужные логарифмы вероятностей
        log_likelihood = np.sum(target * input, axis=1)
        self.output = -np.mean(log_likelihood) # Среднее по батчу
        return self.output

    def updateGradInput(self, input, target):
        batch_size = input.shape[0]
        # gradInput = d(NLL)/d(input) = -1/N * target
        self.gradInput = - target / batch_size
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.